Source: https://www.depends-on-the-definition.com/named-entity-recognition-with-residual-lstm-and-elmo/

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use("ggplot")

#os.chdir(r'D:\My Personal Documents\Learnings\Data Science\Data Sets\NER Dataset')
data = pd.read_csv('df_annotated.csv', encoding="latin1")
data = data.fillna(method="ffill")
data.tail(10)

,Unnamed: 0,Sent,Token,POS,Tag
817637,817637,Sentence:24604,Energy,NNP,O
817638,817638,Sentence:24604,Aneel,NNP,O
817639,817639,Sentence:24604,announced,VBD,O
817640,817640,Sentence:24604,an,DT,O
817641,817641,Sentence:24604,extraordinary,JJ,O
817642,817642,Sentence:24604,increase,NN,O
817643,817643,Sentence:24604,in,IN,O
817644,817644,Sentence:24604,electricity,JJ,O
817645,817645,Sentence:24604,tariffs,NNS,O
817646,817646,Sentence:24604,.,.,O


In [4]:
data.columns = ['Id','Sentence #','Word','POS','Tag' ]

In [20]:
data.head()

,Id,Sentence #,Word,POS,Tag
0,0,Sentence:0,There,EX,O
1,1,Sentence:0,is,VBZ,O
2,2,Sentence:0,a,DT,O
3,3,Sentence:0,small,JJ,O
4,4,Sentence:0,chance,NN,O


In [21]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); 
n_words

30712

In [22]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); 
n_tags

3

In [23]:
tags

['B-org', 'I-org', 'O']

In [24]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [82]:
getter = SentenceGetter(data)

In [83]:
sent = getter.get_next()

In [84]:
sent

In [85]:
sentences = getter.sentences

In [86]:
sentences[9803]

[('Source', 'NN', 'O'),
 ('BofA', 'NNP', 'O'),
 ('Merrill', 'NNP', 'B-org'),
 ('Lynch', 'NNP', 'I-org'),
 ('Global', 'NNP', 'O'),
 ('Research', 'NNP', 'O'),
 ('Bloomberg', 'NNP', 'O'),
 ('GEMs', 'NNP', 'O'),
 ('Daily', 'NNP', 'O'),
 ('Global', 'NNP', 'O'),
 ('Edition', 'NNP', 'O'),
 ('16', 'CD', 'O'),
 ('March', 'NNP', 'O'),
 ('2015', 'CD', 'O'),
 ('8', 'CD', 'O'),
 ('Asia', 'NNP', 'O'),
 ('China', 'NNP', 'O'),
 ('Premier', 'NNP', 'O'),
 ('Li', 'NNP', 'O'),
 ('expressed', 'VBD', 'O'),
 ('confidence', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('delivering', 'VBG', 'O'),
 ('stable', 'JJ', 'O'),
 ('growth', 'NN', 'O'),
 ('Premier', 'NNP', 'O'),
 ('Li', 'NNP', 'O'),
 ('held', 'VBD', 'O'),
 ('a', 'DT', 'O'),
 ('press', 'NN', 'O'),
 ('conference', 'NN', 'O'),
 ('on', 'IN', 'O'),
 ('15', 'CD', 'O'),
 ('March', 'NNP', 'O'),
 ('2015', 'CD', 'O'),
 ('concluding', 'VBG', 'O'),
 ('the', 'DT', 'O'),
 ('annual', 'JJ', 'O'),
 ('NPC', 'NNP', 'O'),
 ('.', '.', 'O')]

In [87]:
max_len = 50
tag2idx = {t: i for i, t in enumerate(tags)}

In [88]:
tag2idx

{'B-org': 0, 'I-org': 1, 'O': 2}

In [89]:
X = [[w[0] for w in s] for s in sentences]

In [90]:
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

In [91]:
X[0]

['There',
 'is',
 'a',
 'small',
 'chance',
 'this',
 'could',
 'abate',
 'slightly',
 'post',
 'elections',
 'but',
 'our',
 'base',
 'case',
 'is',
 'that',
 'another',
 'currency',
 'devaluation',
 'is',
 'needed',
 '.',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__',
 '__PAD__']

In [92]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [93]:
from keras.preprocessing.sequence import pad_sequences
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [94]:
y[1]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2])

In [95]:
from sklearn.model_selection import train_test_split

In [110]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=2018)

In [111]:
batch_size = 32

In [112]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

In [113]:
sess = tf.Session()
K.set_session(sess)

In [114]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [115]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [116]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [117]:
input_text.shape

TensorShape([Dimension(None), Dimension(50)])

In [118]:
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1021 15:10:18.963118 301248 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [119]:
model = Model(input_text, out)

In [120]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [121]:
X_tr, X_val = X_tr[:692*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:692*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

In [122]:
print(y_tr.shape)
print(y_val.shape)
print(len(X_tr))
print(len(X_val))

(22144, 50, 1)
(4320, 50, 1)
22144
4320


In [123]:
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=2, verbose=1)

Train on 22144 samples, validate on 4320 samples
Epoch 1/2


InvalidArgumentError: Incompatible shapes: [1600] vs. [32,50]
	 [[{{node metrics_3/acc/Equal}}]]

In [55]:
hist = pd.DataFrame(history.history)

NameError: name 'history' is not defined

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.title("Learning curves")
plt.legend()
plt.show()

In [ ]:
i = 19
p = model.predict(np.array(X_te[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_te[i], y_te[i], p):
    if w != "__PAD__":
        print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))

In [125]:
sents[0]

NameError: name 'sents' is not defined